In [330]:
import pandas as pd
import matplotlib as plt
import sklearn as sk

In [331]:
df = pd.read_csv('train.csv')

### As we see, There is a lot of useless Data, such as Name, Sex, Etc...

### Pre-Processing :

In [332]:
df

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C


In [333]:
def change_age(age):
    if(age<=10):
        return 'Child'
    elif(10<age<=20):
       return 'teenager'
    elif(20<age<=60):
       return 'Adult'
    elif(age>60):
        return 'Ederly'
        

In [334]:
df['Age']=df['Age'].apply(change_age)

In [335]:
df = df.drop(["Name","Ticket","PassengerId","Fare","Cabin","Embarked","Parch","SibSp"], axis=1)

In [336]:
df.fillna(method='ffill', inplace=True)

In [337]:
df

,Survived,Pclass,Sex,Age
0,0,3,male,Adult
1,1,1,female,Adult
2,1,3,female,Adult
3,1,1,female,Adult
4,0,3,male,Adult
...,...,...,...,...
886,0,2,male,Adult
887,1,1,female,teenager
888,0,3,female,teenager
889,1,1,male,Adult


*Checking For missing values :*

In [338]:
df.isna().sum()

Survived    0
Pclass      0
Sex         0
Age         0
dtype: int64

## Training The data

Let's divite It !

## Random Forest Model

### There is always a peak n_splits where we obtain the most accurate data, let's find out !

In [339]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder

In [340]:
encoder = OneHotEncoder()
encoder2 = LabelEncoder()

In [341]:
df['Sex'] = encoder2.fit_transform(df['Sex'])

In [342]:
encoded_age=encoder.fit_transform(df[['Age']])

In [343]:
encoded_age = pd.DataFrame(encoded_age.toarray(), columns=encoder.get_feature_names_out(['Age']))

In [344]:
encoded_age

,Age_Adult,Age_Child,Age_Ederly,Age_teenager
0,1.0,0.0,0.0,0.0
1,1.0,0.0,0.0,0.0
2,1.0,0.0,0.0,0.0
3,1.0,0.0,0.0,0.0
4,1.0,0.0,0.0,0.0
...,...,...,...,...
886,1.0,0.0,0.0,0.0
887,0.0,0.0,0.0,1.0
888,0.0,0.0,0.0,1.0
889,1.0,0.0,0.0,0.0


In [345]:
df = pd.concat([df, encoded_age], axis=1)

In [346]:
df = df.drop('Age', axis=1)

In [347]:
df

,Survived,Pclass,Sex,Age_Adult,Age_Child,Age_Ederly,Age_teenager
0,0,3,1,1.0,0.0,0.0,0.0
1,1,1,0,1.0,0.0,0.0,0.0
2,1,3,0,1.0,0.0,0.0,0.0
3,1,1,0,1.0,0.0,0.0,0.0
4,0,3,1,1.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...
886,0,2,1,1.0,0.0,0.0,0.0
887,1,1,0,0.0,0.0,0.0,1.0
888,0,3,0,0.0,0.0,0.0,1.0
889,1,1,1,1.0,0.0,0.0,0.0


In [348]:
X = df.drop(["Survived"], axis=1).values
y = df["Survived"].values

In [349]:
df

,Survived,Pclass,Sex,Age_Adult,Age_Child,Age_Ederly,Age_teenager
0,0,3,1,1.0,0.0,0.0,0.0
1,1,1,0,1.0,0.0,0.0,0.0
2,1,3,0,1.0,0.0,0.0,0.0
3,1,1,0,1.0,0.0,0.0,0.0
4,0,3,1,1.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...
886,0,2,1,1.0,0.0,0.0,0.0
887,1,1,0,0.0,0.0,0.0,1.0
888,0,3,0,0.0,0.0,0.0,1.0
889,1,1,1,1.0,0.0,0.0,0.0


In [350]:
from sklearn.model_selection import KFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
model = RandomForestClassifier()

In [351]:
n_splits_range = [2, 3, 4, 15, 20, 50, 100]
acc_list = {}
for n_splits in n_splits_range:
    kf = KFold(n_splits=n_splits)
    avg_acc=[]
    for train_index, test_index in kf.split(X):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]
        model = RandomForestClassifier()
        model.fit(X_train,y_train)
        y_pred = model.predict(X_test)
        acc = accuracy_score(y_test, y_pred)
        avg_acc.append(acc)
        
    average_number = sum(avg_acc)/len(avg_acc)
    acc_list[n_splits] = average_number


max_avg = max(acc_list, key=acc_list.get)
print("Max Average is " ,acc_list[max_avg], "At n_splits =" ,max_avg)




Max Average is  0.7928104575163395 At n_splits = 50


##### Looks like the best score is at n_splits = 15

In [352]:
from sklearn.neighbors import KNeighborsClassifier

In [353]:
n_splits_range = [2, 3, 4, 15, 20, 50, 100]
acc_list = {}
for n_splits in n_splits_range:
    kf = KFold(n_splits=n_splits)
    avg_acc=[]
    for train_index, test_index in kf.split(X):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]
        model = KNeighborsClassifier()
        model.fit(X_train,y_train)
        y_pred = model.predict(X_test)
        acc = accuracy_score(y_test, y_pred)
        avg_acc.append(acc)
        
    average_number = sum(avg_acc)/len(avg_acc)
    acc_list[n_splits] = average_number


max_avg = max(acc_list, key=acc_list.get)
print("Max Average is " ,acc_list[max_avg], "At n_splits =" ,max_avg)



Max Average is  0.7946127946127947 At n_splits = 3


##### Looks like the best score is at n_splits = 60

The best model here seems to be Random Forest at n_splits = 15 !

## Final test

In [354]:
test_data = pd.read_csv('test.csv')
test_data['Age']=test_data['Age'].apply(change_age)
test_data['Sex'] = encoder2.fit_transform(test_data['Sex'])
test_data.fillna(method='ffill', inplace=True)
test_data

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",1,Adult,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",0,Adult,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",1,Ederly,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",1,Adult,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",0,Adult,1,1,3101298,12.2875,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...
413,1305,3,"Spector, Mr. Woolf",1,Adult,0,0,A.5. 3236,8.0500,C78,S
414,1306,1,"Oliva y Ocana, Dona. Fermina",0,Adult,0,0,PC 17758,108.9000,C105,C
415,1307,3,"Saether, Mr. Simon Sivertsen",1,Adult,0,0,SOTON/O.Q. 3101262,7.2500,C105,S
416,1308,3,"Ware, Mr. Frederick",1,Adult,0,0,359309,8.0500,C105,S


In [355]:
encoded_age2=encoder.fit_transform(test_data[['Age']])

In [356]:
encoded_age2 = pd.DataFrame(encoded_age2.toarray(), columns=encoder.get_feature_names_out(['Age']))

In [357]:

test_data=pd.concat([test_data, encoded_age2], axis=1)


In [358]:
test_data

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Age_Adult,Age_Child,Age_Ederly,Age_teenager
0,892,3,"Kelly, Mr. James",1,Adult,0,0,330911,7.8292,NaN,Q,1.0,0.0,0.0,0.0
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",0,Adult,1,0,363272,7.0000,NaN,S,1.0,0.0,0.0,0.0
2,894,2,"Myles, Mr. Thomas Francis",1,Ederly,0,0,240276,9.6875,NaN,Q,0.0,0.0,1.0,0.0
3,895,3,"Wirz, Mr. Albert",1,Adult,0,0,315154,8.6625,NaN,S,1.0,0.0,0.0,0.0
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",0,Adult,1,1,3101298,12.2875,NaN,S,1.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
413,1305,3,"Spector, Mr. Woolf",1,Adult,0,0,A.5. 3236,8.0500,C78,S,1.0,0.0,0.0,0.0
414,1306,1,"Oliva y Ocana, Dona. Fermina",0,Adult,0,0,PC 17758,108.9000,C105,C,1.0,0.0,0.0,0.0
415,1307,3,"Saether, Mr. Simon Sivertsen",1,Adult,0,0,SOTON/O.Q. 3101262,7.2500,C105,S,1.0,0.0,0.0,0.0
416,1308,3,"Ware, Mr. Frederick",1,Adult,0,0,359309,8.0500,C105,S,1.0,0.0,0.0,0.0


In [359]:
kf = KFold(n_splits=15)
for train_index, test_index in kf.split(X):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]
        model_final = KNeighborsClassifier()
        model_final.fit(X_train,y_train)

In [360]:
test_data_feat = test_data.drop(["Name","Ticket","Fare","Cabin","Embarked","Parch","SibSp","PassengerId","Age"], axis=1)
test_data_feat

,Pclass,Sex,Age_Adult,Age_Child,Age_Ederly,Age_teenager
0,3,1,1.0,0.0,0.0,0.0
1,3,0,1.0,0.0,0.0,0.0
2,2,1,0.0,0.0,1.0,0.0
3,3,1,1.0,0.0,0.0,0.0
4,3,0,1.0,0.0,0.0,0.0
...,...,...,...,...,...,...
413,3,1,1.0,0.0,0.0,0.0
414,1,0,1.0,0.0,0.0,0.0
415,3,1,1.0,0.0,0.0,0.0
416,3,1,1.0,0.0,0.0,0.0


In [361]:
df

,Survived,Pclass,Sex,Age_Adult,Age_Child,Age_Ederly,Age_teenager
0,0,3,1,1.0,0.0,0.0,0.0
1,1,1,0,1.0,0.0,0.0,0.0
2,1,3,0,1.0,0.0,0.0,0.0
3,1,1,0,1.0,0.0,0.0,0.0
4,0,3,1,1.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...
886,0,2,1,1.0,0.0,0.0,0.0
887,1,1,0,0.0,0.0,0.0,1.0
888,0,3,0,0.0,0.0,0.0,1.0
889,1,1,1,1.0,0.0,0.0,0.0


In [362]:
predictions = model_final.predict(test_data_feat.values)

In [363]:
final = pd.DataFrame({'PassengerId': test_data.PassengerId, 'Survived': predictions})

In [364]:
final.to_csv('submission2.csv', index=False)